In [1]:
import sys
import cProfile

import numpy as np
import pandas as pd
import seaborn as sns
              
sys.path.insert(0, '../../')
import ccal
%matplotlib inline
%config InlineBackend.figure_formats = {'svg',}

==================== Computational Cancer Analysis Library ====================
<13:47:42> Checking dependencies ...
<13:47:42> Using the following packages:
<13:47:42> 	matplotlib (v1.5.1)
<13:47:42> 	numpy (v1.10.4)
<13:47:42> 	pandas (v0.18.0)
<13:47:42> 	rpy2 (v2.7.9)
<13:47:42> 	scikit-learn (v0.17.1)
<13:47:42> 	scipy (v0.17.0)
<13:47:42> 	seaborn (v0.7.0)


In [2]:
import math

import numpy as np
from scipy.stats import pearsonr
from statsmodels.nonparametric.kernel_density import KDEMultivariate
from scipy.stats import binom_test

# TODO pythonize bcv
import rpy2.robjects as ro
from rpy2.robjects.numpy2ri import numpy2ri

ro.conversion.py2ri = numpy2ri
from rpy2.robjects.packages import importr

mass = importr('MASS')

from ccal.support import drop_nan_columns, add_jitter

def rbcv(x):
    """
    :param x: array-like, (n_samples,)
    :return: float, bandwidth
    """
    bandwidth = np.array(mass.bcv(x))[0]
    return bandwidth

In [ ]:
def information_coefficient(x, y, z=None, n_grid=25, vector_data_types=None, n_perm=0, adaptive=True, alpha=0.05,
                            perm_alpha=0.05):
    """
    :param x: array-like, (n_samples,)
    :param y: array-like, (n_samples,)
    :param z: array-like, (n_samples,), optional, variable on which to condition
    :param n_grid: int, number of grid points at which to evaluate kernel density
    :param vector_data_types: str, 3 chars of 'c' (continuous), 'u' (unordered discrete), or 'o' (ordered discrete)
    :param n_perm: int, >0 will return a p-value in addition to the information coefficient
    :param adaptive: bool, quit permutations after achieving a confidence that the p-value is above (or below) alpha
    :param alpha: float, threshold empirical p-value for significance of IC
    :param perm_alpha: float, threshold probability for terminating adaptive permutation
    :return: float (and float), information coefficient, and the empirical p-value if n_perm > 0
                If adaptive, the accuracy of the empirical p-value will vary: values closer to alpha will be estimated
                more precisely, while values obviously greater or less than alpha will be estimated less precisely.
    """
    vectors = [x, y]
    if z:
        vectors.append(z)
        x, y, z = add_jitter(drop_nan_columns(vectors))
    else:
        x, y = add_jitter(drop_nan_columns(vectors))

    if not vector_data_types:
        # TODO: guess variable types
        vector_data_types = 'c' * len(vectors)
    elif len(vector_data_types) is not len(vectors):
        raise ValueError('Number of specified variable types does not match number of vectors.')

    if len(x) <= len(vector_data_types):
        return 0

    

    mi = mutual_information(x, y, z=z,
                            n_grid=n_grid, vector_data_types=vector_data_types, bandwidth_scaling=bandwidth_scaling)

    ic_sign = np.sign(rho)
    ic = ic_sign * np.sqrt(1 - np.exp(- 2 * mi))

    if n_perm:
        n_more_extreme = 0
        trials = 0
        for i in range(n_perm):
            trials += 1
            # The question is whether I want to have
            # a certain width of confidence interval around my estimate of the pval
            # or just a certain confidence that the pval is greater than 0.05 (current solution)
            pm_x = np.random.permutation(x)
            pm_rho, p = pearsonr(pm_x, y)
            pm_rho2 = abs(pm_rho)
            pm_bandwidth_scaling = (1 + (-0.75) * pm_rho2)
            pm_mi = mutual_information(pm_x, y, z, n_grid=n_grid,
                                       vector_data_types=vector_data_types, bandwidth_scaling=pm_bandwidth_scaling)
            pm_ic_sign = np.sign(pm_rho)
            pm_ic = pm_ic_sign * np.sqrt(1 - np.exp(- 2 * pm_mi))
            if (pm_ic <= ic and ic < 0) or (0 < ic and ic <= pm_ic):
                n_more_extreme += 1
            if adaptive:
                ge_binom_p = binom_test(n_more_extreme, i + 1, alpha, alternative='greater')
                # * 2 because what I'm doing is two-sided testing in both directions
                if ge_binom_p * 2 < perm_alpha:
                    break
                le_binom_p = binom_test(n_more_extreme, i + 1, alpha, alternative='less')
                if le_binom_p * 2 < perm_alpha:
                    break
        p_value = n_more_extreme / float(trials)
        return ic, p_value
    else:
        return ic


# TODO: understand the math
def mutual_information(x, y, z=None, n_grid=25, vector_data_types=None, bandwidth_scaling=None):
    """
    :param x: array-like, (n_samples,)
    :param y: array-like, (n_samples,)
    :param z: array-like, (n_samples,), optional, variable on which to condition
    :param n_grid: int, number of grid points at which to evaluate kernel density
    :param vector_data_types: str, 3 chars of 'c' (continuous), 'u' (unordered discrete), or 'o' (ordered discrete)
    :param bandwidth_scaling: float,
    :return: float, information coefficient
    """
    vectors = [x, y]
    if z:
        vectors.append(z)

    grids = [np.linspace(v.min(), v.max(), n_grid) for v in vectors]
    mesh_grids = np.meshgrid(*grids)
    grid_shape = tuple([n_grid] * len(vectors))
    grid = np.vstack([mesh_grid.flatten() for mesh_grid in mesh_grids])
    delta = np.array([rbcv(q) for q in vectors]).reshape((len(vectors),)) / 4
    if bandwidth_scaling:
        delta *= bandwidth_scaling
    kde = KDEMultivariate(vectors, bw=delta, var_type=vector_data_types)
    p_joint = kde.pdf(grid).reshape(grid_shape) + np.finfo(float).eps
    ds = [grid[1] - grid[0] for grid in grids]
    ds_prod = np.prod(ds)
    p_joint /= (p_joint.sum() * ds_prod)
    h_joint = - np.sum(p_joint * np.log(p_joint)) * ds_prod
    
    dx = ds[0]
    dy = ds[1]
    if z:
        dz = ds[2]
        pxz = p_joint.sum(axis=1) * dy
        pyz = p_joint.sum(axis=0) * dx
        pz = p_joint.sum(axis=(0, 1)) * dx * dy
        hxz = -np.sum(pxz * np.log(pxz)) * dx * dz
        hyz = -np.sum(pyz * np.log(pyz)) * dy * dz
        hz = -np.sum(pz * np.log(pz)) * dz
        cmi = hxz + hyz - h_joint - hz
        return cmi
    else:
        dx = ds[0]
        dy = ds[1]
        px = p_joint.sum(axis=1) * dy
        py = p_joint.sum(axis=0) * dx
        hx = -np.sum(px * np.log(px)) * dx
        hy = -np.sum(py * np.log(py)) * dy
        mi = hx + hy - h_joint
        return mi

In [ ]:
z = None
ngrid = 25
vectors = [np.random.random_sample(1000), np.random.random_sample(1000)]
vector_data_types = 'cc'

if z:
    x, y = vectors
    vectors.append(z)

In [ ]:
vector_grids = [np.linspace(v.min(), v.max(), ngrid) for v in vectors]
vector_grids = np.linspace(0, 10, 5)

In [ ]:
np.meshgrid(*[vector_grids, vector_grids])

In [ ]:
def kde(vectors, vector_data_types, bandwidth_scaling=None):
    bandwidths = np.array([rbcv(v) for v in vectors])
    bandwidths /= 4
    if bandwidth_scaling:
        bandwidths *= bandwidth_scaling
    kde = KDEMultivariate(vectors, vector_data_types, bw=bandwidths)

In [ ]:
def x(vectors):
    delta = np.array([rbcv(z) for z in [x, y]]).reshape((2,)) / 4
    if bandwidth_scaling:
        delta *= bandwidth_scaling
    kde = KDEMultivariate(xy, bw=delta, var_type=var_type)

    fxy = kde.pdf(grid).reshape((n_grid, n_grid)).T + np.finfo(float).eps

In [ ]:


cProfile.run('KDEMultivariate([x,y], bw=[0.06199576, 0.05991138], var_type="cc")', sort=1)

In [ ]:
ns, runtimes = ccal.support.runtime(information_coefficient_old, 1000)
sns.pointplot(ns, runtimes)

In [ ]:
ref = ccal.support.read_gct('/Users/Kwat/Downloads/CCLE.rpkm.v2.SELECTED_SIGNATURES.v2.gct').ix['KRAS_SALE_Late_Comp_C8_9', :]
features = ccal.support.read_gct('/Users/Kwat/Downloads/ccle_mut_cna.gct')
ccal.analyze.rank_features_against_reference(features, ref, nsampling=1,
                                             nperm=1, title='Title', annotation_header='IC' + ' ' * 21 + 'P-val' + ' ' * 4 + 'FDR',
                                             output_prefix='/Users/Kwat/Desktop/ola')

In [ ]:
nrow, ncol = 2, 1000
features = ccal.support.make_random_features(nrow, ncol)
ref = ccal.support.make_random_features(1, ncol)
ccal.analyze.rank_features_against_reference(features, ref, nsampling=2,
                                             nperm=2, title='Title', annotation_header='IC' + ' ' * 21 + 'P-val' + ' ' * 4 + 'FDR',
                                             output_prefix='/Users/Kwat/Desktop/ola')

In [ ]:
ccal.support.VERBOSE = False
for r in range(1, 500, 100):
    for c in range(1, 500, 100):
        features = ccal.support.make_random_features(r, c)
        ref = ccal.support.make_random_features(1, c)
        ccal.analyze.rank_features_against_reference(features, ref, nsampling=2,
                                                     nperm=2, nfeatures=0, title='Continuous {} x {}'.format(r, c))
        
n_category = 10
for r in range(1, 500, 100):
    for c in range(1, 500, 100):
        features = ccal.support.make_random_features(r, c, n_category=n_category)
        ref = ccal.support.make_random_features(1, c, n_category=n_category)
        ccal.analyze.rank_features_against_reference(features, ref, ref_type='categorical',
                                                     nsampling=2, nperm=2, nfeatures=0, title='Categorical {} x {}'.format(r, c))

n_category = 2
for r in range(1, 500, 100):
    for c in range(1, 500, 100):
        features = ccal.support.make_random_features(r, c, n_category=n_category)
        ref = ccal.support.make_random_features(1, c, n_category=n_category)
        ccal.analyze.rank_features_against_reference(features, ref, ref_type='binary',
                                                     nsampling=2, nperm=2, nfeatures=0, title='Binary {} x {}'.format(r, c))

# Test IC

In [ ]:
x = np.random.random_sample(10)
y = np.random.random_sample(10)
print(ccal.information.information_coefficient(x, y))

x = np.random.random_sample(10)
y = np.random.random_sample(11)
try:
    ccal.information.information_coefficient(x, y)
except ValueError as e:
    print(e)

x = np.random.random_sample(10)
x[1] = None
y = np.random.random_sample(10)
y[2] = None
print(ccal.information.information_coefficient(x, y))

x = np.random.random_sample(10)
x[1] = None
y = np.random.random_sample(10)
y[2] = None
y[6] = None
print(ccal.information.information_coefficient(x, y))

x = np.random.random_sample(10)
x[1] = None
x[3] = None
x[5] = None
y = np.random.random_sample(10)
y[2] = None
y[4] = None
print(ccal.information.information_coefficient(x, y))

x = np.array([12.517, 14.706, np.nan, 14.12, np.nan, np.nan, np.nan, 12.255])
y = np.array([0.98246356, 0.97525171, 0.77744759, 0.64084311, 0.4405853, 0.43827196, 0.12447757, 0.08116039])
print(ccal.information.information_coefficient(x, y))

# Make simulation matrix

In [ ]:
def add_value(df, inVal, outVal):
    # Add value in cluster
    # Set inVal or outVal to be None when not updating it
    
    for i,(n,s) in enumerate(df.iterrows()):
        #print('add_value:',i)
        for j,c in enumerate(s.index):
            if inVal and n==c:
                df.iloc[i,j]=inVal
            if outVal and n!=c:
                df.iloc[i,j]=outVal

def add_noise(df,inMu,inSigma,outMu,outSigma):
    # Add noise
    
    for i,(n,s) in enumerate(df.iterrows()):
        #print('add_noise:',i)
        for j,c in enumerate(s.index):
            if (inMu or inSigma) and n==c:
                df.iloc[i,j]+=random.gauss(inMu,inSigma)
            if (outMu or outSigma) and n!=c:
                df.iloc[i,j]+=random.gauss(outMu,outSigma)

def mix(df,k,mix):
    # Mix cluster values to nonclusters and vice versa
    
    assert k!=0, print('k cannot be 0')
    
    # Get the number of values in a cluster
    n_k_val=len(df.columns)*len(df.index)/k
    #print('n_k_val:',n_k_val)
    
    # Count
    c=0
    while c<mix*n_k_val:
        
        # Pick 1st random index and column
        r_idx0=random.randint(0,len(df.index)-1)
        r_col0=random.randint(0,len(df.columns)-1)

        # If index and column locate inside a cluster
        if df.index[r_idx0]==df.columns[r_col0]:    
            
            # Get cluster value located
            pick0=df.iloc[r_idx0,r_col0]
            
            # Pick 2nd random index and column
            r_idx1=random.randint(0,len(df.index)-1)
            r_col1=random.randint(0,len(df.columns)-1)
            
            # If index and column locate outside a cluster
            if df.index[r_idx1]!=df.columns[r_col1]:    

                # Get non-cluster value located
                pick1=df.iloc[r_idx1,r_col1]

                # Swap
                df.iloc[r_idx0,r_col0]=pick1
                df.iloc[r_idx1,r_col1]=pick0
                
                # Count
                c+=1
                
                #print('Swapped (%s,%s) & (%s,%s)'%(df.index[r_idx0],df.columns[r_col0],df.index[r_idx1],df.columns[r_col1]))

def initialize_simulation_df(df,inVal,inMu,inSigma,outVal,outMu,outSigma,mix):
    # Initialize values in and out of a cluster and add noise
    
    t0=time()
    
    # For each row
    for i,(n,s) in enumerate(df.iterrows()):
        print('initialize_simulation_df:',i)
        
        # For each column
        for j,c in enumerate(s.index):
            r=random.random()
            
            if mix and r<=mix:
                # Mix
                if n==c:
                    # In cluster gets out-value
                    df.iloc[i,j]=outVal
                    if outMu or outSigma:
                        df.iloc[i,j]+=random.gauss(outMu,outSigma)
                else:
                    # Out cluster gets in-value
                    df.iloc[i,j]=inVal
                    if inMu or inSigma:
                        df.iloc[i,j]+=random.gauss(inMu,inSigma) 
            else:
                # No mix
                if n==c:
                    # In cluster gets in-value
                    df.iloc[i,j]=inVal
                    if inMu or inSigma:
                        df.iloc[i,j]+=random.gauss(inMu,inSigma)
                else:
                    # Out cluster gets out-value
                    df.iloc[i,j]=outVal
                    if outMu or outSigma:
                        df.iloc[i,j]+=random.gauss(outMu,outSigma)
    print('initialize_simulation_df: done in %0.3fs.'%(time()-t0))

def plot_mtrx(mtrx):
    # Plot simulation matrix
    
    plt.imshow(mtrx,interpolation='nearest',cmap=plt.cm.ocean)
    plt.colorbar()
    plt.show()
    
def make_mtrx_sample_x_variable_simulation(n_sample,
                                           n_var,
                                           k,
                                           val_in,
                                           val_out,
                                           noise_in_mu=None,
                                           noise_in_sigma=None,
                                           noise_out_mu=None,
                                           noise_out_sigma=None,
                                           noise_mix=None,
                                           prefix_out_f=None,
                                           suffix_out_f=None,
                                           plot=False):
    """
    Make sample x variable matrix
    """
    assert k != 0, 'k cannot be 0'
    assert k <= n_var,'k cannot be greater than n_var'
    
    # Make an empty sample x variable matrix filled with 0
    mtrx_sample_x_var = pd.DataFrame(index=range(n_sample), columns=range(n_var)).fillna(0)

    # Slice dataframe index and column and make lists of dataframe indexes and columns for each index and column slice
    list_index_slice = toolK.slice_list(mtrx_sample_x_var.index, k)
    list_column_slice = toolK.slice_list(mtrx_sample_x_var.columns, k)

    # Make index and column slice x dataframe indexes dictionaries
    dict_index_slice = {}
    for i, l in enumerate(list_index_slice):
        dict_index_slice[i] = l
    dict_column_slice = {}
    for i, l in enumerate(list_column_slice):
        dict_column_slice[i] = l

    # Set dataframe index and column to be index and column slice indices respectively
    index=list(mtrx_sample_x_var.index)
    for i,l in dict_index_slice.items():
        for j in l:
            index[j]=i
    mtrx_sample_x_var.index=index
    columns=list(mtrx_sample_x_var.columns)
    for i,l in dict_column_slice.items():
        for j in l:
            columns[j]=i
    mtrx_sample_x_var.columns=columns

    # Initialize simulation matrix
    initialize_simulation_df(mtrx_sample_x_var,val_in,noise_in_mu,noise_in_sigma,val_out,noise_out_mu,noise_out_sigma,noise_mix)
    
    # Save
    if prefix_out_f:        
        mtrx_sample_x_var.to_csv(prefix_out_f+'_sample_%s_var_%s_k_%s_mix_%s_%s'%(n_sample,n_var,k,noise_mix,suffix_out_f),sep='\t')
    
    # Plot
    if plot:
        plot_mtrx(mtrx_sample_x_var)

# Make simulation matrix

# Set number of samples
list_n_sample = [100, 500, 1000, 5000]
# Set number of variables
list_n_var = [100, 500, 1000, 5000]
# Set values in clusters
val_in = 1
# Set values out of clusters
val_out = 0
# Set Ks
list_k = [1, 2, 3, 4, 5, 6, 10, 15, 20, 25]
# Set the fractions of cluster values to be swapped between clusters and nonclusters
list_noise_mix = [0, 0.05, 0.1, 0.2]
# Set noise in clusters
noise_in_mu = 0
noise_in_sigma = 0.1 * noise_in_mu
# Set noise out of clusters
noise_out_mu = 0
noise_out_sigma = 0.1*noise_out_mu

# Simulate
for sample in list_n_sample:
    print('sample:',sample)
    
    for var in list_n_var:
        print('\tvar:',var)
        
        for k in list_k:
            print('\t\tk:',k)
            
            for noise_mix in list_noise_mix:
                print('\t\t\tnoise_mix:',noise_mix)
                
                make_mtrx_sample_x_variable_simulation(sample,
                                                       var,
                                                       k,
                                                       val_in,
                                                       val_out,
                                                       noise_mix=noise_mix,
                                                       prefix_out_f='/cellar/users/hyeerna/aLL/mtrx_sample_x_var_simulation/test/',
                                                       suffix_out_f='sfx',
                                                       plot=True)